# Tensor Network solver for QUBO, QUDO and Tensor QUDO problems in a linear chain with one-neighbor interaction

In this notebook we will solve the n-neighbor Tensor QUDO problem defined as a minimization problem with integer $x_i$ between $0$ and $D_i-1$, where the cost function is:

$$C(\vec{x})=\sum_{i=0}^{N-2}W_{i,i+1,x_{i},x_{i+1}}+\sum_{i=0}^{N-1}W_{i,i,x_{i},x_{i}}$$

The interaction will only be between variables $i$ and $i+1$. That is, the tensor $W_{i,j,k,l}$ will be an sparse tensor which satisfies that $j=i$ or $j=i+1$. For simplicity, all variables have the same dimensionality.

For application cases, we implement also the particular case for QUDO problems

$$C(\vec{x})=\sum_{i=0}^{N-2}W_{i,i+1}x_{i}x_{i+1}+\sum_{i=0}^{N-1}(W_{i,i}x_{i}^2 + d_i x_{i})$$

where $W$ is a tridiagonal matrix.

In [1]:
# Import required libraries
# Tensor Network algorithm
import numpy as np  # For numerical computations and array operations
from decimal import Decimal  # For precise decimal arithmetic

# Comparative
from ortools.sat.python import cp_model  # Google OR-Tools constraint programming solver
import dimod  # For quantum-inspired optimization

# Experiments
from time import perf_counter  # For performance timing
from tqdm import tqdm  # For progress bars
import matplotlib.pyplot as plt  # For plotting results

# Configure matplotlib plot styling
plt.rcParams.update({
    'font.size': 15,
    'axes.labelsize': 15,
    'xtick.labelsize': 15,
    'ytick.labelsize': 15,
    'legend.fontsize': 15
})


---
# Auxiliary functions

## Computation functions

In [2]:
def cost_QUDO(w_matrix: np.ndarray, d_vector: np.ndarray, solution: list) -> float:
    '''
    Calculate the cost of a solution for a Quadratic Unconstrained Discrete Optimization (QUDO) problem.
    
    The cost function is defined as:
    C(x) = sum(W[i,j] * x[i] * x[j]) + sum(d[i] * x[i])
    where W is a symmetric weight matrix and d is a vector of linear coefficients.
    
    Args:
        w_matrix: Weight matrix of shape (N,N), symmetric matrix containing quadratic terms
        d_vector: Vector of shape (N,) containing linear coefficients
        solution: Solution vector of shape (N,) containing integer values
    
    Returns:
        float: Cost value for the given solution
    '''
    _cost = 0
    n_variables = len(solution)
    for i in range(n_variables):
        _cost += d_vector[i] * solution[i]
        for j in range(i, n_variables):
            _cost += w_matrix[i,j] * solution[i] * solution[j]
    return _cost
        
def last_variable_determination_QUDO(solution: list, w_matrix: np.ndarray,
                                    d_vector: np.ndarray, dimensions: np.ndarray) -> int:
    '''
    Determine optimal value for last variable given values of all others in a QUDO problem.
    
    For a partial solution with N-1 variables set, this function evaluates all possible values
    for the Nth variable and returns the value that minimizes the total cost function.
    
    Args:
        solution: Current partial solution with N-1 variables set
        w_matrix: Weight matrix of shape (N,N) containing quadratic terms
        d_vector: Vector of shape (N,) containing linear coefficients
        dimensions: Array of dimensions for each variable, where dimensions[-1] is the 
                   number of possible values for the last variable
        
    Returns:
        int: Optimal value (0 to dimensions[-1]-1) for last variable that minimizes total cost
    '''
    _cost_list = np.zeros(dimensions[-1])
    for _valor in range(dimensions[-1]):
        _cost_list[_valor] = cost_QUDO(w_matrix, d_vector, solution[:-1] + [_valor])
    return np.argmin(_cost_list)



def cost_tensor_QUDO(w_tensor: np.ndarray, solution: list) -> float:
    '''
    Calculate the cost of a solution for a Tensor Quadratic Unconstrained Discrete Optimization (T-QUDO) problem.
    
    The cost function is defined as:
    C(x) = sum(W[i,j,x[i],x[j]]) 
    where W is a 4D tensor containing the cost coefficients for each combination of variable values.
    
    Args:
        w_tensor: 4D tensor of shape (N,N,D,D) where N is number of variables and D is max dimension.
                 w_tensor[i,j,xi,xj] gives cost coefficient for variables i,j taking values xi,xj.
        solution: List of N integers representing the solution vector, where solution[i] is the value
                 assigned to variable i.
    
    Returns:
        float: Total cost value for the given solution
    '''
    _cost = 0
    n_variables = len(solution)
    for i in range(n_variables):
        for j in range(i, n_variables):
            _cost += w_tensor[i,j,solution[i],solution[j]]
    return _cost
        
def last_variable_determination_tensor_QUDO(solution: list, w_tensor: np.ndarray) -> int:
    '''
    Determine optimal value for last variable given values of all others in a Tensor QUDO problem.
    
    For a partial solution with N-1 variables set, this function evaluates all possible values
    for the Nth variable and returns the value that minimizes the total cost function.
    The cost is calculated using the tensor representation where costs are stored directly for
    each combination of variable values.
    
    Args:
        solution: List of N-1 integers representing current partial solution with last variable unset
        w_tensor: 4D tensor of shape (N,N,D,D) containing cost coefficients for all variable value combinations
        
    Returns:
        int: Optimal value (0 to dimensions-1) for last variable that minimizes total cost
    '''
    dimensions = w_tensor.shape[-1]
    _cost_list = np.zeros(dimensions)
    for _valor in range(dimensions):
        _cost_list[_valor] = cost_tensor_QUDO(w_tensor, solution[:-1] + [_valor])
    return np.argmin(_cost_list)

## Test functions

In [3]:
def generate_QUDO_instance(n_variables: int, values_range: tuple) -> tuple[np.ndarray, np.ndarray]:
    """
    Generates a random symmetric QUDO instance matrix with nearest-neighbor interactions.
    
    Args:
        n_variables: Number of variables in the QUDO instance
        values_range: Tuple of (min, max) values for matrix elements
        
    Returns:
        tuple: (W_matrix, d_vector)
            - W_matrix: Symmetric matrix with random values in the specified range
            - d_vector: Vector of linear terms with random values in the specified range
    """
    if not isinstance(n_variables, int) or n_variables < 1:
        raise ValueError("n_variables must be a positive integer")
    if not isinstance(values_range, tuple) or len(values_range) != 2 or values_range[1] <= values_range[0]:
        raise ValueError("values_range must be a tuple of (min, max) with max > min")
        
    w_matrix = np.zeros((n_variables, n_variables), dtype=float)
    min_val, max_val = values_range
    
    # Generate random linear terms more efficiently using vectorized operations
    d_vector = np.random.uniform(min_val, max_val, n_variables)
    
    # Generate random quadratic terms with nearest-neighbor interactions
    for i in range(n_variables):
        for j in range(i, min(i+2, n_variables)):
            w_matrix[i,j] = np.random.uniform(min_val, max_val, 1)[0]
            w_matrix[j,i] = w_matrix[i,j]  # Ensure symmetry

    
    w_matrix = w_matrix / np.linalg.norm(w_matrix)  # Normalize interaction matrix
    d_vector = d_vector / np.linalg.norm(d_vector)  # Normalize linear terms
    
    return w_matrix, d_vector

def generate_tensor_QUDO_instance(n_variables: int, dimensions: int, values_range: tuple) -> np.ndarray:
    """
    Generates a random Tensor QUDO instance tensor with nearest-neighbor interactions.
    
    Args:
        n_variables: Number of variables in the QUDO instance
        dimensions: Dimension of each variable's state space
        values_range: Tuple of (min, max) values for tensor elements
        
    Returns:
        np.ndarray: 4D tensor of shape (n_variables, n_variables, dimensions, dimensions) 
                   containing the interaction terms between variables
    """
    if not isinstance(n_variables, int) or n_variables < 1:
        raise ValueError("n_variables must be a positive integer")
    if not isinstance(dimensions, int) or dimensions < 1:
        raise ValueError("dimensions must be a positive integer")
    if not isinstance(values_range, tuple) or len(values_range) != 2 or values_range[1] <= values_range[0]:
        raise ValueError("values_range must be a tuple of (min, max) with max > min")
        
    w_tensor = np.zeros((n_variables, n_variables, dimensions, dimensions), dtype=float)
    min_val, max_val = values_range
    
    # Generate random quadratic terms with nearest-neighbor interactions
    for i in range(n_variables):
        for j in range(i, min(i+2, n_variables)):
            w_tensor[i,j] = np.random.uniform(min_val, max_val, (dimensions, dimensions))

    
    w_tensor = w_tensor / np.linalg.norm(w_tensor)  # Normalize interaction matrix
    
    return w_tensor


def solve_QUDO_ortools(w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, max_time: float) -> list:
    """
    Solves a QUDO problem using OR-Tools with a time limit.
    Always returns a solution, even if not optimal.
    
    Args:
        w_matrix: Symmetric matrix with quadratic terms
        d_vector: Vector with linear terms
        dimensions: Vector with the dimension of each variable
        max_time: Maximum time in seconds to spend solving
        
    Returns:
        list: Solution vector (may be non-optimal if time limit is reached)
    """
    t_0 = perf_counter()
    model = cp_model.CpModel()
    
    n_variables = len(dimensions)
    x = [model.NewIntVar(0, dim-1, f'x{i}') for i, dim in enumerate(dimensions)]
    
    objective = 0
    
    # Process quadratic terms more efficiently
    for i in range(n_variables):
        # Add diagonal terms
        if w_matrix[i,i] != 0:
            aux = model.NewIntVar(0, (dimensions[i]-1)**2, f'aux_{i}_{i}')
            model.AddMultiplicationEquality(aux, x[i], x[i])
            objective += w_matrix[i,i] * aux
        
        # Add off-diagonal terms only for non-zero coefficients
        for j in range(i+1, n_variables):
            if abs(w_matrix[i,j]) > 1e-14:  # Use small threshold for floating point comparison
                aux = model.NewIntVar(0, (dimensions[i]-1)*(dimensions[j]-1), f'aux_{i}_{j}')
                model.AddMultiplicationEquality(aux, x[i], x[j])
                objective += w_matrix[i,j] * aux
    
    # Add linear terms more efficiently
    non_zero_indices = np.nonzero(d_vector)[0]
    for i in non_zero_indices:
        objective += d_vector[i] * x[i]
        
    model.Minimize(objective)
    
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = max_time
    status = solver.Solve(model)
    
    if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
        return [solver.Value(x[i]) for i in range(n_variables)]
    
    return [0] * n_variables


def solve_QUBO_dimod(w_matrix: np.ndarray) -> list:
    """
    Solve binary QUBO problem using dimod's SimulatedAnnealingSampler.
    Optimized for linear chain problems with improved parameter selection.
    
    Args:
        w_matrix: Quadratic terms matrix
    
    Returns:
        list: Binary solution vector (0 or 1 values)
    """
    # Create QUBO dictionary more efficiently
    Q = {(i,j): w_matrix[i,j] 
         for i in range(len(w_matrix)) 
         for j in range(i, len(w_matrix)) 
         if abs(w_matrix[i,j]) > 1e-10}
    
    bqm = dimod.BinaryQuadraticModel.from_qubo(Q)
    sampler = dimod.SimulatedAnnealingSampler()
    
    # Optimized parameters for better exploration and convergence
    sampleset = sampler.sample(bqm,
                             num_reads=10,  # Increased reads for better sampling
                             num_sweeps=10,  # Increased sweeps for thorough exploration
                             beta_range=(0.001, 100.0))  # Wider temperature range
    
    # Get best solution
    return list(sampleset.first.sample.values())


# Solve using different methods and compare results
def print_solution(name, solution, time, cost):
    print(f"\n{name} Results:")
    print(f"Solution time: {time:.6e} seconds")
    print(f"Solution: {','.join(map(str, solution))}")
    print(f"Solution cost: {cost:.6e}")


---
# QUDO Solver

## Definition of the tensor network nodes

In [4]:
def tensor_initial_generator_QUDO(position: int, solution: list, w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, tau: float, phase: bool=False) -> np.ndarray:
    """
    Generate initial tensor for QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        solution (list): Partial solution vector
        w_matrix (ndarray): Weight matrix of shape (N,N) containing quadratic interactions
        d_vector (ndarray): Linear term vector of length N containing linear costs
        dimensions (np.ndarray): List of dimensions for each variable
        tau (float): Evolution parameter for optimization
        phase (bool): Whether to use phase factors in tensor elements
        
    Returns:
        ndarray: Initial tensor of shape (dimensions[position], dimensions[position])
    """
    # Initialize tensor with appropriate dtype
    dtype = complex if phase else float
    tensor = np.zeros((dimensions[position], dimensions[position]), dtype=dtype)
    phase_factor = 1j*2*np.pi/dimensions[position] if phase else 0
    
    # Calculate tensor elements
    for i in range(dimensions[position]):
        # Compute total energy contribution
        exp_value = (d_vector[position]*i +                    # Linear term
                    w_matrix[position, position]*i*i)          # Self-interaction
        
        # Add interaction with previous variable if not first position
        if position > 0:  # Check solution is not empty
            exp_value += w_matrix[position-1, position]*solution[position-1]*i
            
        # Set diagonal element with evolution factor
        tensor[i,i] = np.exp(-tau*exp_value) * np.exp(phase_factor*i)
    
    # Normalize tensor    
    norm = np.linalg.norm(tensor)
    return tensor / (norm if norm > 0 else 1.0)

def tensor_intermediate_generator_QUDO(position: int, w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, tau: float, phase: bool=False) -> np.ndarray:
    """
    Generate intermediate tensor for QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        w_matrix (ndarray): Weight matrix of shape (N,N) containing quadratic interactions
        d_vector (ndarray): Linear term vector of length N containing linear costs
        dimensions (np.ndarray): List of dimensions for each variable
        tau (float): Evolution parameter for optimization
        phase (bool): Whether to use phase factors in tensor elements
        
    Returns:
        ndarray: Intermediate tensor of shape (dimensions[position-1], dimensions[position])
    """
    # Initialize tensor with appropriate dtype
    dtype = complex if phase else float
    tensor = np.zeros((dimensions[position-1], dimensions[position]), dtype=dtype)
    phase_factor = 1j*2*np.pi/dimensions[position] if phase else 0
    
    # Calculate tensor elements
    for i in range(dimensions[position-1]):
        for j in range(dimensions[position]):
            # Compute total energy contribution
            exp_value = (d_vector[position]*j +                    # Linear term
                        w_matrix[position, position]*j*j +         # Self-interaction
                        w_matrix[position-1, position]*i*j)        # Interaction with previous
                
            # Set element with evolution factor
            tensor[i,j] = np.exp(-tau*exp_value) * np.exp(phase_factor*j)

    # Normalize tensor
    norm = np.linalg.norm(tensor)
    return tensor / (norm if norm > 0 else 1.0)

def tensor_final_generator_QUDO(position: int, w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, tau: float, phase: bool=False) -> np.ndarray:
    """
    Generate final tensor for QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        w_matrix (ndarray): Weight matrix of shape (N,N) containing quadratic interactions
        d_vector (ndarray): Linear term vector of length N containing linear costs
        dimensions (np.ndarray): List of dimensions for each variable
        tau (float): Evolution parameter for optimization
        phase (bool): Whether to use phase factors in tensor elements
        
    Returns:
        ndarray: Final tensor of shape (dimensions[position-1])
    """
    # Initialize tensor with appropriate dtype
    dtype = complex if phase else float
    tensor = np.zeros(dimensions[position-1], dtype=dtype)
    phase_factor = 1j*2*np.pi/dimensions[position] if phase else 0
    
    # Calculate tensor elements by summing over last variable
    for i in range(dimensions[position-1]):
        for j in range(dimensions[position]):
            # Compute total energy contribution
            exp_value = (d_vector[position]*j +                    # Linear term
                        w_matrix[position, position]*j*j +         # Self-interaction
                        w_matrix[position-1, position]*i*j)        # Interaction with previous
                
            # Add contribution to element with evolution factor
            tensor[i] += np.exp(-tau*exp_value) * np.exp(phase_factor*j)

    # Normalize tensor
    norm = np.linalg.norm(tensor)
    return tensor / (norm if norm > 0 else 1.0)

## Generate Tensor network

In [5]:
def generate_tensor_network_QUDO(w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, tau: float, phase: bool = False) -> list[np.ndarray]:
    """
    Generates a tensor network for solving a Quadratic Unconstrained Discrete Optimization problem.
    
    Args:
        w_matrix (np.ndarray): Square matrix representing quadratic interactions between variables
        d_vector (np.ndarray): Vector representing linear costs
        dimensions (np.ndarray): Array containing the dimension of each variable
        tau (float): Imaginary time evolution parameter for optimization
        phase (bool): Whether to include phase factors in tensor elements
        
    Returns:
        list[np.ndarray]: List of tensors representing the tensor network, from left to right
    """
    # Input validation
    if w_matrix.shape[0] != w_matrix.shape[1]:
        raise ValueError("Weight matrix must be square")
    if len(d_vector) != len(w_matrix):
        raise ValueError("Dimension of d_vector must match w_matrix")
    if len(dimensions) != len(w_matrix):
        raise ValueError("Length of dimensions must match number of variables")
        
    n_variables = len(w_matrix)
    tensor_list = []
    
    # Create initial measurement tensor
    tensor_list.append(tensor_initial_generator_QUDO(0, [], w_matrix, d_vector, dimensions, tau, phase))

    # Create intermediate tensors 
    for position in range(1, n_variables-1):
        tensor_list.append(tensor_intermediate_generator_QUDO(position, w_matrix, d_vector, dimensions, tau, phase))
    
    # Create final tensor
    tensor_list.append(tensor_final_generator_QUDO(n_variables-1, w_matrix, d_vector, dimensions, tau, phase))

    return tensor_list

## Contraction scheme

In [6]:
def contraction_tensors(tensor_list: list[np.ndarray]) -> list[np.ndarray]:
    """
    Contracts a list of tensors sequentially and returns intermediate results.
    
    Args:
        tensor_list: List of tensors to contract. Each tensor should be a numpy array.
        
    Returns:
        List of intermediate contracted tensors, normalized at each step.
        The first element is the rightmost tensor, and subsequent elements
        are the results of contracting from right to left.
    """
    if not tensor_list:
        return []
        
    # Initialize with last tensor normalized
    vector = tensor_list[-1].copy()  # Make copy to avoid modifying input
    norm = np.linalg.norm(vector)
    if norm > 0:  # Only normalize if norm is not zero
        vector /= norm
    intermediate_tensors = [vector]
    
    # Contract tensors from right to left
    for tensor in reversed(tensor_list[:-1]):
        # Contract and normalize
        vector = tensor @ vector
        norm = np.linalg.norm(vector)
        if norm > 0:  # Avoid division by zero
            vector /= norm
        intermediate_tensors.append(vector)

    return list(reversed(intermediate_tensors))

## Global QUDO Solver Function

Function that receives tridiagonal matrix $W$, the lineal cost vector $d$ the dimension of the different variables and the minimization $\tau$, and returns the result as a vector.

$x_i\in [0,D_i-1]$

In [7]:
def tn_qudo_solver(w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, tau: float = 1, phase: bool = False) -> list[int]:
    """
    Solves a Quadratic Unconstrained Discrete Optimization problem using tensor networks.
    
    Args:
        w_matrix: Square matrix representing quadratic interactions between variables.
                 Must be symmetric and have shape (n_variables, n_variables).
        d_vector: Vector representing linear costs. Must have length n_variables.
        dimensions: Array containing the dimension of each variable.
                   Must have length n_variables and contain positive integers.
        tau: Imaginary time for optimization (default=1). Controls optimization strength.
        phase: If True, uses phase-based optimization (humbucker method).
             If False, uses standard amplitude-based optimization.
        
    Returns:
        List of integers containing the optimal values for each variable,
        where the i-th value is in range [0, dimensions[i]-1].
        
    Raises:
        ValueError: If input dimensions don't match or are invalid.
    """
    # Validate inputs
    n_variables = len(w_matrix)
    if w_matrix.shape != (n_variables, n_variables):
        raise ValueError("w_matrix must be square")
    if len(d_vector) != n_variables:
        raise ValueError("d_vector length must match w_matrix dimensions")
    if len(dimensions) != n_variables:
        raise ValueError("dimensions length must match number of variables")
    if not np.all(dimensions > 0):
        raise ValueError("All dimensions must be positive integers")
        
    solution = [0] * n_variables

    # Create and contract the tensor network
    tensor_network = generate_tensor_network_QUDO(w_matrix, d_vector, dimensions, tau, phase)
    intermediate_tensors = contraction_tensors(tensor_network)

    # Get first variable value and clean up tensors
    solution[0] = int(np.argmax(np.abs(intermediate_tensors[0])))
    intermediate_tensors = intermediate_tensors[2:]  # Remove first two tensors

    # Iteratively determine remaining variables
    for position in range(1, n_variables-1):
        # Scale tau proportionally as we progress through variables
        proportion = n_variables / (n_variables - position)
        scaled_tau = tau * proportion
        
        # Generate and contract tensors for current position
        initial_tensor = tensor_initial_generator_QUDO(
            position, solution, w_matrix, d_vector, dimensions, scaled_tau, phase
        )
        output_vector = initial_tensor @ intermediate_tensors[0]
        intermediate_tensors.pop(0)

        # Select optimal value based on maximum probability amplitude
        solution[position] = int(np.argmax(np.abs(output_vector)))

    # Determine final variable value
    solution[-1] = last_variable_determination_QUDO(solution, w_matrix, d_vector, dimensions)

    return solution

# QUDO Test

## Experiment function

In [8]:
def run_qudo_experiments(n_vars_range: tuple[int,int,int], dims_range: tuple[int,int,int], tau_range: tuple[float,float,int]):
    """
    Run experiments comparing different aspects of QUDO solver performance
    and generate plots for analysis.
    
    Args:
        n_vars_range: Tuple of (start, stop, step) for number of variables to test
        dims_range: Tuple of (start, stop, step) for dimensions to test
        tau_range: Tuple of (start, stop, num_points) for tau values to test
        
    Generates three plots:
    - Time vs number of variables for different dimensions
    - Time vs dimension for different numbers of variables  
    - Approximation ratio vs tau for different numbers of variables
    """
    VALUES_RANGE = (-1, 1)
    
    # Generate parameter ranges
    n_vars = np.arange(*n_vars_range, dtype=int)
    n_dims = np.arange(*dims_range, dtype=int)
    n_taus = np.linspace(*tau_range, dtype=float)
    n_taus = 10**n_taus

    # Experiment 1 & 2: Runtime analysis
    times_tensor = np.zeros((len(n_vars), len(n_dims)))
    
    total_iterations = len(n_vars) * len(n_dims)
    with tqdm(total=total_iterations, desc="Running experiments") as pbar:
        for n_idx, n_variables in enumerate(n_vars):
            for d_idx, d in enumerate(n_dims):
                # Scale tau inversely with problem size
                tau = 1e2 / n_variables / d
                dimensions = d * np.ones(n_variables, dtype=int)
                
                # Generate random problem
                w_matrix, d_vector = generate_QUDO_instance(int(n_variables), VALUES_RANGE)
                if d == 2:  # Binary case
                    d_vector = np.zeros(n_variables)
                    
                # Time tensor network solution
                start = perf_counter()
                _ = tn_qudo_solver(w_matrix, d_vector, dimensions, tau, phase=False)
                times_tensor[n_idx, d_idx] = perf_counter() - start
                
                pbar.update(1)
    # Plot and save time vs variables
    plt.figure(figsize=(10, 6))
    plt.xlabel('Number of Variables')
    plt.ylabel('Time (seconds)')
    for d_idx, d in enumerate(n_dims):
        plt.plot(n_vars, times_tensor[:,d_idx], 'o-', label=f'D={d}')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/time_vs_variables_qudo.pdf')
    plt.close()
    
    # Save time vs variables data
    np.save('results/time_vs_variables_qudo.npy', times_tensor)

    # Plot and save time vs dimension
    plt.figure(figsize=(10, 6))
    plt.xlabel('Dimension of Variables')
    plt.ylabel('Time (seconds)')
    for n_idx, n_variables in enumerate(n_vars[::len(n_vars)//10]):
        plt.plot(n_dims, times_tensor[n_idx*(len(n_vars)//10)], 'o-', label=f'N={n_variables}')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/time_vs_dimension_qudo.pdf')
    plt.close()
    
    # Save time vs dimension data
    np.save('results/time_vs_dimension_qudo.npy', times_tensor)

    # Experiment 3: Approximation ratio analysis
    FIXED_DIM = 2
    N_VALUES = 10
    fixed_n_vars = np.arange(5,5+N_VALUES)
    aprox_tensor = np.zeros((N_VALUES, len(n_taus)))
    total_iterations = N_VALUES * len(n_taus)
    with tqdm(total=total_iterations, desc="Calculating approximation ratios") as pbar:
        for n_idx, n_variables in enumerate(fixed_n_vars):
            for tau_idx, tau in enumerate(n_taus):
                dimensions = FIXED_DIM * np.ones(n_variables, dtype=int)
                
                # Generate random problem
                w_matrix, d_vector = generate_QUDO_instance(int(n_variables), VALUES_RANGE)
                d_vector = np.zeros(n_variables)  # Binary case
                
                # Compare solutions
                solution_tn = tn_qudo_solver(w_matrix, d_vector, dimensions, tau, phase=False)
                solution_dimod = solve_QUBO_dimod(w_matrix)
                
                # Calculate approximation ratio
                cost_tn = cost_QUDO(w_matrix, d_vector, solution_tn)
                cost_dimod = cost_QUDO(w_matrix, d_vector, solution_dimod)
                aprox_tensor[n_idx, tau_idx] = cost_tn / cost_dimod
                
                pbar.update(1)
            
    # Plot and save approximation ratio vs tau
    plt.figure(figsize=(10, 6))
    plt.xlabel(r'$\tau$')
    plt.ylabel('Cost Ratio (Tensor/Dimod)')
    for n_idx, n_variables in enumerate(fixed_n_vars):
        plt.semilogx(n_taus, aprox_tensor[n_idx], 'o-', label=f'N={n_variables}')
    plt.yscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/ratio_vs_tau_qudo.pdf')
    plt.close()
    
    # Save approximation ratio data
    np.save('results/ratio_vs_tau_qudo.npy', aprox_tensor)


## Experiments

In [9]:
# Experiment parameters
np.random.seed(123)
n_variables = 50
dimensions = 2 * np.ones(n_variables, dtype=int)
VALUES_RANGE = (-1.0, 1.0)

# Optimization parameters
tau = 1e3 / n_variables / dimensions[0]  # Scale tau inversely with problem size
phase = False

# Generate and normalize QUDO instance
w_tensor, d_vector = generate_QUDO_instance(n_variables, VALUES_RANGE)

# For binary problems, set linear terms to zero
if dimensions[0] == 2:
    d_vector = np.zeros(n_variables)

# ---------------------------------------------------
# Tensor Network solution
start_time = perf_counter()
solution_TN = tn_qudo_solver(w_tensor, d_vector, dimensions, tau=tau, phase=phase)
tn_time = perf_counter() - start_time
print_solution("Tensor Network", solution_TN, tn_time, 
              cost_QUDO(w_tensor, d_vector, solution_TN))

# OR-Tools solution
start_time = perf_counter()
solution_OR = solve_QUDO_ortools(w_tensor, d_vector, dimensions, tn_time)
or_time = perf_counter() - start_time
print_solution("OR-Tools", solution_OR, or_time,
              cost_QUDO(w_tensor, d_vector, solution_OR))

# Dimod solution (only for binary problems)
if dimensions[0] == 2:
    start_time = perf_counter()
    solution_dimod = solve_QUBO_dimod(w_tensor)
    dimod_time = perf_counter() - start_time
    print_solution("Dimod", solution_dimod, dimod_time,
                  cost_QUDO(w_tensor, d_vector, solution_dimod))



Tensor Network Results:
Solution time: 1.500400e-03 seconds
Solution: 1,0,1,1,0,0,0,0,0,1,1,0,1,1,1,0,0,0,1,1,0,1,0,1,1,1,1,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,1,1,1,0,1,0,0
Solution cost: -2.299585e+00

OR-Tools Results:
Solution time: 6.917900e-03 seconds
Solution: 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Solution cost: 0.000000e+00

Dimod Results:
Solution time: 7.582600e-03 seconds
Solution: 1,0,1,1,0,0,0,1,1,1,1,0,1,1,1,0,0,0,1,1,0,1,0,1,1,0,1,1,0,0,1,1,0,0,1,0,1,0,1,0,0,0,0,1,1,1,0,1,0,0
Solution cost: -2.327419e+00


In [10]:
# Set up experiment parameters
n_vars_range = (5,1050,20)
dims_range = (2,16,3)
tau_range = (0.1, 2.5, 10)

# Run the experiment
run_qudo_experiments(n_vars_range, dims_range, tau_range)

Calculating approximation ratios:   0%|          | 0/100 [00:00<?, ?it/s]C:\Users\aleja\AppData\Local\Temp\ipykernel_24276\3081164278.py:96: RuntimeWarning: invalid value encountered in scalar divide
  aprox_tensor[n_idx, tau_idx] = cost_tn / cost_dimod
Calculating approximation ratios: 100%|██████████| 100/100 [00:00<00:00, 615.19it/s]


---
# Tensor QUDO Solver

## Definition of the tensor network nodes

In [11]:
def tensor_initial_generator_tensor_QUDO(position: int, solution: list, w_tensor: np.ndarray, tau: float, phase: bool=False) -> np.ndarray:
    """
    Generate initial tensor for T-QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        solution (list): Partial solution vector
        w_tensor (ndarray): Weight tensor of shape (N,N,D,D) containing quadratic interactions
        tau (float): Evolution parameter for optimization
        phase (bool): Whether to use phase factors in tensor elements
        
    Returns:
        ndarray: Initial tensor of shape (dimensions[position], dimensions[position])
    """
    # Initialize tensor with appropriate dtype
    dtype = complex if phase else float
    dimensions = w_tensor.shape[2]
    tensor = np.zeros((dimensions, dimensions), dtype=dtype)
    phase_factor = 1j*2*np.pi/dimensions if phase else 0
    
    # Calculate tensor elements
    for i in range(dimensions):
        # Compute total energy contribution
        exp_value = w_tensor[position, position, i, i]  # Self-interaction
        
        # Add interaction with previous variable if not first position
        if position > 0:  # Check solution is not empty
            exp_value += w_tensor[position-1, position, solution[position-1], i]
            
        # Set diagonal element with evolution factor
        tensor[i,i] = np.exp(-tau*exp_value) * np.exp(phase_factor*i)
    
    # Normalize tensor    
    norm = np.linalg.norm(tensor)
    return tensor / (norm if norm > 0 else 1.0)

def tensor_intermediate_generator_tensor_QUDO(position: int, w_tensor: np.ndarray, tau: float, phase: bool=False) -> np.ndarray:
    """
    Generate intermediate tensor for T-QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        w_tensor (ndarray): Weight tensor of shape (N,N,D,D) containing quadratic interactions
        tau (float): Evolution parameter for optimization
        phase (bool): Whether to use phase factors in tensor elements
        
    Returns:
        ndarray: Intermediate tensor of shape (dimensions[position-1], dimensions[position])
    """
    # Initialize tensor with appropriate dtype
    dtype = complex if phase else float
    dimensions = w_tensor.shape[2]
    tensor = np.zeros((dimensions, dimensions), dtype=dtype)
    phase_factor = 1j*2*np.pi/dimensions if phase else 0
    
    # Calculate tensor elements
    for i in range(dimensions):
        for j in range(dimensions):
            # Compute total energy contribution
            exp_value = (w_tensor[position, position, j, j] +         # Self-interaction
                        w_tensor[position-1, position, i, j])        # Interaction with previous
                
            # Set element with evolution factor
            tensor[i,j] = np.exp(-tau*exp_value) * np.exp(phase_factor*j)

    # Normalize tensor
    norm = np.linalg.norm(tensor)
    return tensor / (norm if norm > 0 else 1.0)

def tensor_final_generator_tensor_QUDO(position: int, w_tensor: np.ndarray, tau: float, phase: bool=False) -> np.ndarray:
    """
    Generate final tensor for T-QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        w_tensor (ndarray): Weight tensor of shape (N,N,D,D) containing quadratic interactions
        tau (float): Evolution parameter for optimization
        phase (bool): Whether to use phase factors in tensor elements
        
    Returns:
        ndarray: Final tensor of shape (dimensions[position-1])
    """
    # Initialize tensor with appropriate dtype
    dimensions = w_tensor.shape[2]
    dtype = complex if phase else float
    tensor = np.zeros(dimensions, dtype=dtype)
    phase_factor = 1j*2*np.pi/dimensions if phase else 0
    
    # Calculate tensor elements by summing over last variable
    for i in range(dimensions):
        for j in range(dimensions):
            # Compute total energy contribution
            exp_value = (w_tensor[position, position, j, j] +         # Self-interaction
                        w_tensor[position-1, position, i, j])        # Interaction with previous
                
            # Add contribution to element with evolution factor
            tensor[i] += np.exp(-tau*exp_value) * np.exp(phase_factor*j)

    # Normalize tensor
    norm = np.linalg.norm(tensor)
    return tensor / (norm if norm > 0 else 1.0)

## Generate Tensor network

In [12]:
def generate_tensor_network_tensor_QUDO(w_tensor: np.ndarray, tau: float, phase: bool = False) -> list[np.ndarray]:
    """
    Generates a tensor network for solving a Tensor Quadratic Unconstrained Discrete Optimization problem.
    
    Args:
        w_tensor (np.ndarray): Tensor representing quadratic interactions between variables
        tau (float): Imaginary time evolution parameter for optimization
        phase (bool): Whether to include phase factors in tensor elements
        
    Returns:
        list[np.ndarray]: List of tensors representing the tensor network, from left to right
    """
    n_variables = len(w_tensor)
    tensor_list = []
    
    # Create initial measurement tensor
    tensor_list.append(tensor_initial_generator_tensor_QUDO(0, [], w_tensor, tau, phase))

    # Create intermediate tensors 
    for position in range(1, n_variables-1):
        tensor_list.append(tensor_intermediate_generator_tensor_QUDO(position, w_tensor, tau, phase))
    
    # Create final tensor
    tensor_list.append(tensor_final_generator_tensor_QUDO(n_variables-1, w_tensor, tau, phase))

    return tensor_list

## Global Tensor QUDO Solver Function

Function that receives tensor $W$, the dimension of the different variables and the minimization $\tau$, and returns the result as a vector.

$x_i\in [0,D_i-1]$

In [13]:
def tn_qudo_solver_tensor(w_tensor: np.ndarray, tau: float = 1, phase: bool = False) -> list[int]:
    """
    Solves a Tensor Quadratic Unconstrained Discrete Optimization problem using tensor networks.
    
    Args:
        w_tensor: Tensor representing quadratic interactions between variables.
                 Must have shape (n_variables, n_variables).
        tau: Imaginary time for optimization (default=1). Controls optimization strength.
        phase: If True, uses phase-based optimization (humbucker method).
             If False, uses standard amplitude-based optimization.
        
    Returns:
        List of integers containing the optimal values for each variable.
        
    Raises:
        ValueError: If input dimensions don't match or are invalid.
    """
    n_variables = len(w_tensor)
    solution = [0] * n_variables

    # Create and contract the tensor network
    tensor_network = generate_tensor_network_tensor_QUDO(w_tensor, tau, phase)
    intermediate_tensors = contraction_tensors(tensor_network)

    # Get first variable value and clean up tensors
    solution[0] = int(np.argmax(np.abs(intermediate_tensors[0])))
    intermediate_tensors = intermediate_tensors[2:]  # Remove first two tensors

    # Iteratively determine remaining variables
    for position in range(1, n_variables-1):
        # Scale tau proportionally as we progress through variables
        proportion = n_variables / (n_variables - position)
        scaled_tau = tau * proportion
        
        # Generate and contract tensors for current position
        initial_tensor = tensor_initial_generator_tensor_QUDO(
            position, solution, w_tensor, scaled_tau, phase
        )
        output_vector = initial_tensor @ intermediate_tensors[0]
        intermediate_tensors.pop(0)

        # Select optimal value based on maximum probability amplitude
        solution[position] = int(np.argmax(np.abs(output_vector)))

    # Determine final variable value
    solution[-1] = last_variable_determination_tensor_QUDO(solution, w_tensor)

    return solution

# Tensor QUDO Test

## Experiment function

In [14]:
def run_tensor_qudo_experiments(n_vars_range: tuple[int,int,int], dims_range: tuple[int,int,int]):
    """
    Run experiments comparing different aspects of QUDO solver performance
    and generate plots for analysis.
    
    Args:
        n_vars_range: Tuple of (start, stop, step) for number of variables to test
        dims_range: Tuple of (start, stop, step) for dimensions to test
        
    Generates two plots:
    - Time vs number of variables for different dimensions
    - Time vs dimension for different numbers of variables
    """
    VALUES_RANGE = (-1, 1)
    
    # Generate parameter ranges
    n_vars = np.arange(*n_vars_range, dtype=int)
    n_dims = np.arange(*dims_range, dtype=int)

    # Runtime analysis
    times_tensor = np.zeros((len(n_vars), len(n_dims)))
    
    total_iterations = len(n_vars) * len(n_dims)
    with tqdm(total=total_iterations, desc="Running experiments") as pbar:
        for n_idx, n_variables in enumerate(n_vars):
            for d_idx, d in enumerate(n_dims):
                # Scale tau inversely with problem size
                tau = 1e2 / n_variables / d
                
                # Generate random problem
                w_matrix = generate_tensor_QUDO_instance(int(n_variables), int(d), VALUES_RANGE)
                
                # Time tensor network solution
                start = perf_counter()
                _ = tn_qudo_solver_tensor(w_matrix, tau, phase=False)
                times_tensor[n_idx, d_idx] = perf_counter() - start
                
                pbar.update(1)

    # Plot and save time vs variables
    plt.figure(figsize=(10, 6))
    plt.xlabel('Number of Variables')
    plt.ylabel('Time (seconds)')
    for d_idx, d in enumerate(n_dims):
        plt.plot(n_vars, times_tensor[:,d_idx], 'o-', label=f'D={d}')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/time_vs_variables_tensor_qudo.pdf')
    plt.close()
    
    # Save time vs variables data
    np.save('results/time_vs_variables_tensor_qudo.npy', times_tensor)

    # Plot and save time vs dimension
    plt.figure(figsize=(10, 6))
    plt.xlabel('Dimension of Variables')
    plt.ylabel('Time (seconds)')
    for n_idx, n_variables in enumerate(n_vars[::len(n_vars)//10]):
        plt.plot(n_dims, times_tensor[n_idx*(len(n_vars)//10)], 'o-', label=f'N={n_variables}')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/time_vs_dimension_tensor_qudo.pdf')
    plt.close()
    
    # Save time vs dimension data
    np.save('results/time_vs_dimension_tensor_qudo.npy', times_tensor)


## Experiments

In [15]:
# Experiment parameters
np.random.seed(123)
n_variables = 500
dimensions = 2
VALUES_RANGE = (-1.0, 1.0)

# Optimization parameters
tau = 1e4 / n_variables / dimensions  # Scale tau inversely with problem size
phase = False

# Generate and normalize QUDO instance
w_tensor = generate_tensor_QUDO_instance(n_variables, dimensions, VALUES_RANGE)

# ---------------------------------------------------
# Tensor Network solution
start_time = perf_counter()
solution_TN = tn_qudo_solver_tensor(w_tensor, tau=tau, phase=phase)
tn_time = perf_counter() - start_time
print_solution("Tensor Network", solution_TN, tn_time, 
              cost_tensor_QUDO(w_tensor, solution_TN))




Tensor Network Results:
Solution time: 4.429550e-02 seconds
Solution: 0,0,1,1,0,0,1,0,1,0,1,0,0,0,1,0,1,1,1,0,1,1,1,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,1,1,1,0,1,0,1,0,1,1,1,1,0,1,0,0,0,1,1,1,1,0,0,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,1,1,1,0,0,1,1,1,0,1,1,0,0,1,1,0,1,0,1,0,0,0,1,0,1,1,0,1,1,1,1,0,1,1,1,1,1,0,0,0,1,1,0,1,0,1,0,1,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,1,0,0,1,0,0,1,1,1,1,1,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,1,0,0,0,0,0,0,1,0,1,0,1,0,1,0,1,1,1,1,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,1,0,1,1,0,0,0,0,0,1,1,1,0,1,0,0,0,1,1,1,1,1,1,1,1,1,1,0,0,0,0,1,0,1,1,0,1,1,0,1,1,1,0,1,0,0,1,1,1,1,0,0,0,0,1,0,1,1,0,0,1,0,0,1,1,1,0,0,1,0,0,1,0,0,1,1,0,1,1,1,0,1,0,0,0,0,1,0,1,1,0,1,0,0,1,1,0,1,1,1,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,1,1,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,1,0,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,1,1,1,0,0,1,0,0,0,1,0,0,1,0,1,1,1,0,1,0,0,0,1,0,0,0,1,1,1,0,1,1,1,1,1,1,0,0,0,1,0,1,0,0,1,0,1,0,1,0,0,0,0,0,0,1,1,1,1,1,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,1,0,0,0,0

In [16]:
# Set up experiment parameters
n_vars_range = (5,1050,20)
dims_range = (2,16,3)

# Run the experiment
run_tensor_qudo_experiments(n_vars_range, dims_range)

Running experiments: 100%|██████████| 265/265 [01:25<00:00,  3.09it/s]


---
# QUDO Solver with Decimal

## Definition of the tensor network nodes

In [17]:
def tensor_initial_generator_QUDO_decimal(position: int, solution: list, w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: list, tau: float) -> list:
    """
    Generate initial tensor for QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        solution (list): Partial solution vector
        w_matrix (np.ndarray): Weight matrix of shape (N,N) containing quadratic interactions
        d_vector (np.ndarray): Linear term vector of length N containing linear costs
        dimensions (list): List of dimensions for each variable
        tau (float): Evolution parameter for optimization
        
    Returns:
        list: Initial tensor of shape (dimensions[position], dimensions[position]), normalized
    """
    # Initialize tensor with zeros
    tensor = [[Decimal('0') for _ in range(dimensions[position])] for _ in range(dimensions[position])]

    # Calculate tensor elements
    for i in range(dimensions[position]):
        # Compute total energy contribution
        exp_value = (d_vector[position] * Decimal(str(i)) +                    # Linear term
                    w_matrix[position][position] * Decimal(str(i**2)))         # Self-interaction
        
        # Add interaction with previous variable if not first position
        if position >= 1:
            exp_value += w_matrix[position-1][position] * Decimal(str(solution[position-1])) * Decimal(str(i))
            
        # Set diagonal element with evolution factor
        tensor[i][i] = (-tau * exp_value).exp()
    
    # Efficient norm calculation for diagonal matrix
    norm = sum(elem * elem for row in tensor for elem in row).sqrt()
    
    # Normalize tensor elements in place
    return [[elem/norm for elem in row] for row in tensor]

def tensor_intermediate_generator_QUDO_decimal(position: int, w_matrix: list, d_vector: list, dimensions: list, tau: Decimal) -> list:
    """
    Generate intermediate tensor for QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        w_matrix (list): Weight matrix of shape (N,N) containing quadratic interactions
        d_vector (list): Linear term vector of length N containing linear costs
        dimensions (list): List of dimensions for each variable
        tau (Decimal): Evolution parameter for optimization
        
    Returns:
        list: Normalized intermediate tensor of shape (dimensions[position-1], dimensions[position])
    """
    # Initialize tensor with zeros
    tensor = [[Decimal('0') for _ in range(dimensions[position])] for _ in range(dimensions[position-1])]
    
    # Calculate tensor elements
    for j in range(dimensions[position]):
        for i in range(dimensions[position-1]):
            # Compute total energy contribution
            exp_value = (d_vector[position] * Decimal(str(j)) +                    # Linear term
                        w_matrix[position][position] * Decimal(str(j**2)) +        # Self-interaction
                        w_matrix[position-1][position] * Decimal(str(i*j)))        # Interaction with previous
                
            # Set element with evolution factor
            tensor[i][j] = (-tau * exp_value).exp()

    norm = sum(elem * elem for row in tensor for elem in row).sqrt()
    return [[elem/norm for elem in row] for row in tensor]

def tensor_final_generator_QUDO_decimal(position: int, w_matrix: list, d_vector: list, dimensions: list, tau: Decimal) -> list:
    """
    Generate final tensor for QUDO problem at given position.
    
    Args:
        position (int): Current position in solution vector
        w_matrix (list): Weight matrix of shape (N,N) containing quadratic interactions
        d_vector (list): Linear term vector of length N containing linear costs
        dimensions (list): List of dimensions for each variable
        tau (Decimal): Evolution parameter for optimization
        
    Returns:
        list: Normalized final tensor of shape (dimensions[position-1])
    """
    # Initialize tensor with zeros
    tensor = [Decimal('0') for _ in range(dimensions[position-1])]
    
    # Calculate tensor elements by summing over last variable
    for j in range(dimensions[position]):
        for i in range(dimensions[position-1]):
            # Compute total energy contribution
            exp_value = (d_vector[position] * Decimal(str(j)) +                    # Linear term
                        w_matrix[position][position] * Decimal(str(j**2)) +        # Self-interaction
                        w_matrix[position-1][position] * Decimal(str(i*j)))        # Interaction with previous
                
            # Add contribution to element with evolution factor
            tensor[i] += (-tau * exp_value).exp()

    norm = sum(elem * elem for elem in tensor).sqrt()
    return [elem/norm for elem in tensor]

## Generate Tensor network

In [18]:
def generate_tensor_network_QUDO_decimal(w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: list, tau: Decimal) -> list:
    """
    Generates a tensor network for solving a Quadratic Unconstrained Discrete Optimization problem.
    
    Args:
        w_matrix: Square matrix representing quadratic interactions between variables
        d_vector: Vector representing linear costs
        dimensions: List containing the dimension of each variable
        tau: Imaginary time for optimization
        
    Returns:
        List of tensors representing the tensor network
    """
    n_variables = len(w_matrix)
    tensor_list = []
    
    # Create initial measurement tensor
    tensor_list.append(tensor_initial_generator_QUDO_decimal(0, [], w_matrix, d_vector, dimensions, tau))

    # Create intermediate tensors 
    for position in range(1, n_variables-1):
        tensor_list.append(tensor_intermediate_generator_QUDO_decimal(position, w_matrix, d_vector, dimensions, tau))
    
    # Create final tensor
    tensor_list.append(tensor_final_generator_QUDO_decimal(n_variables-1, w_matrix, d_vector, dimensions, tau))

    return tensor_list

## Contraction scheme

In [19]:
def contraction_tensors_decimal(tensor_list: list) -> list:
    """
    Contracts a list of tensors sequentially and returns intermediate results.
    
    Args:
        tensor_list: List of tensors to contract
        
    Returns:
        List of intermediate contracted tensors, normalized at each step
    """
    if not tensor_list:
        return []
        
    # Initialize with last tensor
    vector = tensor_list[-1]
    intermediate_tensors = [vector.copy()]
    
    # Contract tensors from right to left
    for tensor in reversed(tensor_list[:-1]):
        # Contract tensors
        output_vector = [sum(tensor[i][j] * vector[j] for j in range(len(vector))) 
                        for i in range(len(tensor))]
        
        # Normalize using decimal
        norm = sum(elem * elem for elem in output_vector).sqrt()
        vector = [elem/norm for elem in output_vector]
        
        intermediate_tensors.append(vector)

    return list(reversed(intermediate_tensors))

---
## Global QUDO Solver Function

Function that receives tridiagonal matrix $W$, the lineal cost vector $d$ the dimension of the different variables and the minimization $\tau$, and returns the result as a vector.

$x_i\in [0,D_i-1]$

In [20]:
def tn_qudo_solver_decimal(w_matrix: np.ndarray, d_vector: np.ndarray, dimensions: np.ndarray, tau: float = 1) -> list:
    """
    Solves a Quadratic Unconstrained Discrete Optimization problem using tensor networks.
    
    Args:
        w_matrix: Square matrix representing quadratic interactions between variables
        d_vector: Vector representing linear costs
        dimensions: Array containing the dimension of each variable
        tau: Imaginary time for optimization (default=1)
        
    Returns:
        List containing the optimal values for each variable
    """
    n_variables = len(w_matrix)
    solution = [0] * n_variables
    
    # Convert inputs to Decimal for higher precision
    w_matrix_decimal = [[Decimal(str(elem)) for elem in row] for row in w_matrix]
    d_vector_decimal = [Decimal(str(elem)) for elem in d_vector]
    tau_decimal = Decimal(str(tau))

    # Create and contract the tensor network
    tensor_network = generate_tensor_network_QUDO_decimal(w_matrix_decimal, d_vector_decimal, dimensions, tau_decimal)
    intermediate_tensors = contraction_tensors_decimal(tensor_network)
    
    # Get first variable value from initial tensor contraction
    solution[0] = int(np.argmax([abs(elem) for elem in intermediate_tensors[0]]))
    
    # Remove first two tensors as they're no longer needed
    intermediate_tensors = intermediate_tensors[2:]

    # Iteratively determine remaining variables except the last one
    for position in range(1, n_variables - 1):
        # Calculate proportion for dynamic tau scaling
        proportion = Decimal(str(n_variables / (n_variables - position)))
        scaled_tau = tau_decimal * proportion
        
        # Generate tensor for current position
        initial_tensor = tensor_initial_generator_QUDO_decimal(position, solution, w_matrix_decimal, 
                                                             d_vector_decimal, dimensions, scaled_tau)
        
        # Contract with remaining intermediate tensors
        output_vector = [
            sum(initial_tensor[i][j] * intermediate_tensors[0][j] 
                for j in range(len(intermediate_tensors[0])))
            for i in range(len(initial_tensor))
        ]
        
        # Remove used intermediate tensor
        intermediate_tensors.pop(0)

        # Select optimal value based on maximum absolute value
        solution[position] = int(np.argmax([abs(elem) for elem in output_vector]))

    # Determine final variable value
    solution[-1] = last_variable_determination_QUDO(solution, w_matrix, d_vector, dimensions)

    return solution

# QUDO Decimal Test

## Experiment function

In [21]:
def run_qudo_experiments_decimal(n_vars_range: tuple[int,int,int], dims_range: tuple[int,int,int], tau_range: tuple[float,float,int]):
    """
    Run experiments comparing different aspects of QUDO solver performance
    and generate plots for analysis.
    
    Args:
        n_vars_range: Tuple of (start, stop, step) for number of variables to test
        dims_range: Tuple of (start, stop, step) for dimensions to test
        tau_range: Tuple of (start, stop, num_points) for tau values to test
        
    Generates three plots:
    - Time vs number of variables for different dimensions
    - Time vs dimension for different numbers of variables  
    - Approximation ratio vs tau for different numbers of variables
    """
    VALUES_RANGE = (-1, 1)
    
    # Generate parameter ranges
    n_vars = np.arange(*n_vars_range, dtype=int)
    n_dims = np.arange(*dims_range, dtype=int)
    n_taus = np.linspace(*tau_range, dtype=float)
    n_taus = 10**n_taus

    # Experiment 1 & 2: Runtime analysis
    times_tensor = np.zeros((len(n_vars), len(n_dims)))
    
    total_iterations = len(n_vars) * len(n_dims)
    with tqdm(total=total_iterations, desc="Running experiments") as pbar:
        for n_idx, n_variables in enumerate(n_vars):
            for d_idx, d in enumerate(n_dims):
                # Scale tau inversely with problem size
                tau = 1e4 / n_variables / d
                dimensions = d * np.ones(n_variables, dtype=int)
                
                # Generate random problem
                w_matrix, d_vector = generate_QUDO_instance(int(n_variables), VALUES_RANGE)
                if d == 2:  # Binary case
                    d_vector = np.zeros(n_variables)
                    
                # Time tensor network solution
                start = perf_counter()
                _ = tn_qudo_solver_decimal(w_matrix, d_vector, dimensions, tau)
                times_tensor[n_idx, d_idx] = perf_counter() - start
                
                pbar.update(1)
    # Plot and save time vs variables
    plt.figure(figsize=(10, 6))
    plt.xlabel('Number of Variables')
    plt.ylabel('Time (seconds)')
    for d_idx, d in enumerate(n_dims):
        plt.plot(n_vars, times_tensor[:,d_idx], 'o-', label=f'D={d}')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/time_vs_variables_qudo_decimal.pdf')
    plt.close()
    
    # Save time vs variables data
    np.save('results/time_vs_variables_qudo_decimal.npy', times_tensor)

    # Plot and save time vs dimension
    plt.figure(figsize=(10, 6))
    plt.xlabel('Dimension of Variables')
    plt.ylabel('Time (seconds)')
    for n_idx, n_variables in enumerate(n_vars[::len(n_vars)//10]):
        plt.plot(n_dims, times_tensor[n_idx*(len(n_vars)//10)], 'o-', label=f'N={n_variables}')
    plt.legend()
    plt.savefig('figures/time_vs_dimension_qudo_decimal.pdf')
    plt.close()
    
    # Save time vs dimension data
    np.save('results/time_vs_dimension_qudo_decimal.npy', times_tensor)

    # Experiment 3: Approximation ratio analysis
    FIXED_DIM = 2
    N_VALUES = 10
    fixed_n_vars = np.arange(5,5+N_VALUES)
    aprox_tensor = np.zeros((N_VALUES, len(n_taus)))
    total_iterations = N_VALUES * len(n_taus)
    with tqdm(total=total_iterations, desc="Calculating approximation ratios") as pbar:
        for n_idx, n_variables in enumerate(fixed_n_vars):
            for tau_idx, tau in enumerate(n_taus):
                dimensions = FIXED_DIM * np.ones(n_variables, dtype=int)
                
                # Generate random problem
                w_matrix, d_vector = generate_QUDO_instance(int(n_variables), VALUES_RANGE)
                d_vector = np.zeros(n_variables)  # Binary case
                
                # Compare solutions
                solution_tn = tn_qudo_solver_decimal(w_matrix, d_vector, dimensions, tau)
                solution_dimod = solve_QUBO_dimod(w_matrix)
                
                # Calculate approximation ratio
                cost_tn = cost_QUDO(w_matrix, d_vector, solution_tn)
                cost_dimod = cost_QUDO(w_matrix, d_vector, solution_dimod)
                aprox_tensor[n_idx, tau_idx] = cost_tn / cost_dimod
                
                pbar.update(1)
            
    # Plot and save approximation ratio vs tau
    plt.figure(figsize=(10, 6))
    plt.xlabel(r'$\tau$')
    plt.ylabel('Cost Ratio (Tensor/Dimod)')
    for n_idx, n_variables in enumerate(fixed_n_vars):
        plt.semilogx(n_taus, aprox_tensor[n_idx], 'o-', label=f'N={n_variables}')
    plt.yscale('log')
    plt.legend()
    plt.tight_layout()
    plt.savefig('figures/ratio_vs_tau_qudo_decimal.pdf')
    plt.close()
    
    # Save approximation ratio data
    np.save('results/ratio_vs_tau_qudo_decimal.npy', aprox_tensor)


## Experiments

In [22]:
# Experiment parameters
np.random.seed(123)
n_variables = 10
dimensions = 2 * np.ones(n_variables, dtype=int)
VALUES_RANGE = (-1.0, 1.0)

# Optimization parameters
tau = 1e5 / n_variables  # Scale tau inversely with problem size
phase = False

# Generate and normalize QUDO instance
w_tensor, d_vector = generate_QUDO_instance(n_variables, VALUES_RANGE)

# For binary problems, set linear terms to zero
if dimensions[0] == 2:
    d_vector = np.zeros(n_variables)

# ---------------------------------------------------
# Tensor Network solution
start_time = perf_counter()
solution_TN = tn_qudo_solver_decimal(w_tensor, d_vector, dimensions, tau=tau)
tn_time = perf_counter() - start_time
print_solution("Tensor Network", solution_TN, tn_time, 
              cost_QUDO(w_tensor, d_vector, solution_TN))

# OR-Tools solution
start_time = perf_counter()
solution_OR = solve_QUDO_ortools(w_tensor, d_vector, dimensions, tn_time)
or_time = perf_counter() - start_time
print_solution("OR-Tools", solution_OR, or_time,
              cost_QUDO(w_tensor, d_vector, solution_OR))

# Dimod solution (only for binary problems)
if dimensions[0] == 2:
    start_time = perf_counter()
    solution_dimod = solve_QUBO_dimod(w_tensor)
    dimod_time = perf_counter() - start_time
    print_solution("Dimod", solution_dimod, dimod_time,
                  cost_QUDO(w_tensor, d_vector, solution_dimod))


Tensor Network Results:
Solution time: 7.175000e-04 seconds
Solution: 0,1,1,1,1,0,0,0,1,1
Solution cost: -1.251903e+00

OR-Tools Results:
Solution time: 1.889600e-03 seconds
Solution: 0,0,0,0,0,0,0,0,0,0
Solution cost: 0.000000e+00

Dimod Results:
Solution time: 1.734900e-03 seconds
Solution: 0,1,1,1,1,0,0,0,1,1
Solution cost: -1.251903e+00


In [23]:
# Set up experiment parameters
n_vars_range = (5,1050,20)
dims_range = (2,16,3)
tau_range = (0.1, 5, 10)

# Run the experiment
run_qudo_experiments_decimal(n_vars_range, dims_range, tau_range)

Calculating approximation ratios: 100%|██████████| 100/100 [00:00<00:00, 507.62it/s]
